In [16]:
#Conexion a la base de datos AdventureWorks en SQL Server database
import pyodbc
import pandas as pd
import numpy as np 
from datetime import datetime

connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"  # O el driver que tengas instalado
    "SERVER=ADRIANAC\SQLEXPRESS;" #Nombre o direccion IP del SQL server
    "DATABASE=AdventureWorks2022;" #Nombre de la base de datos que tenog en SQL y quiero usar
    "Trusted_Connection=yes;" #Porque tengo autenticacion de windows, sino, me pediria usuario y psw
)

# Establecer la conexión
with pyodbc.connect(connection_string) as conn:
    # Crear el cursor
    cursor = conn.cursor()


<>:9: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\S'
C:\Users\adria\AppData\Local\Temp\ipykernel_10176\2190362467.py:9: SyntaxWarning: invalid escape sequence '\S'
  "SERVER=ADRIANAC\SQLEXPRESS;" #Nombre o direccion IP del SQL server


In [17]:
#Extrae los datos de la tabla Person.Person.
#Primero realizamos la consulta para identificar los duplicados que tengamos
query = """
SELECT *
FROM Person.Person
"""
# Imprimir la consulta
#print(query)
# Leer datos en un DataFrame de Pandas
df = pd.read_sql(query, conn)
# Mostrar las primeras filas
print(df.head(5).to_markdown(index=False, numalign="left", stralign="left"))

C:\Users\adria\AppData\Local\Temp\ipykernel_10176\4114558389.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


| BusinessEntityID   | PersonType   | NameStyle   | Title   | FirstName   | MiddleName   | LastName   | Suffix   | EmailPromotion   | AdditionalContactInfo   | Demographics                                                                                                                                                        | rowguid                              | ModifiedDate        |
|:-------------------|:-------------|:------------|:--------|:------------|:-------------|:-----------|:---------|:-----------------|:------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------|:--------------------|
| 1                  | EM           | False       |         | Ken         | J            | Sánchez    |          | 0                |                         | <IndividualSurvey xmlns="http://schemas.microsoft.com/sqlserver/20

In [22]:
# Consulta SQL para extraer los datos
query = "SELECT BusinessEntityID, ModifiedDate FROM Person.Person"

# Leer datos en un DataFrame de Pandas
df = pd.read_sql(query, conn)

# Convertir ModifiedDate a tipo datetime
df['ModifiedDate'] = pd.to_datetime(df['ModifiedDate'])

# Mostrar las primeras filas
print(df.head(5).to_markdown(index=False, numalign="left", stralign="left"))

# Calcular la antigüedad en días
today = pd.to_datetime(datetime.today())
df['AntiguedadDias'] = (today - df['ModifiedDate']).dt.days

# Calcular estadísticas descriptivas
estadisticas = df['AntiguedadDias'].describe()
percentiles = df['AntiguedadDias'].quantile([0.25, 0.75])

# Mostrar los resultados
print("Estadísticas de Antigüedad (en días):")
print(estadisticas)
print(f"  Percentil 25: {percentiles[0.25]:.0f}")
print(f"  Percentil 75: {percentiles[0.75]:.0f}")

| BusinessEntityID   | ModifiedDate        |
|:-------------------|:--------------------|
| 1                  | 2009-01-07 00:00:00 |
| 2                  | 2008-01-24 00:00:00 |
| 3                  | 2007-11-04 00:00:00 |
| 4                  | 2007-11-28 00:00:00 |
| 5                  | 2007-12-30 00:00:00 |
Estadísticas de Antigüedad (en días):
count    19972.000000
mean      4098.565141
std        383.883572
min       3401.000000
25%       3842.000000
50%       3981.000000
75%       4256.000000
max       6620.000000
Name: AntiguedadDias, dtype: float64
  Percentil 25: 3842
  Percentil 75: 4256


C:\Users\adria\AppData\Local\Temp\ipykernel_10176\2378593986.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [27]:
#Agrupa los datos por SalesOrderID y suma la columna TotalLinea para obtener el total de cada orden.
# Consulta SQL para extraer los datos
query = "SELECT SalesOrderID, LineTotal FROM Sales.SalesOrderDetail"

# Leer datos en un DataFrame de Pandas
df = pd.read_sql(query, conn)

# Mostrar las primeras filas
print(df.head(5).to_markdown(index=False, numalign="left", stralign="left"))

# Agrupar por 'SalesOrderID' y sumar 'TotalLinea'
grouped_df = df.groupby('SalesOrderID')['LineTotal'].sum().reset_index()

# Renombrar la columna resultante para mayor claridad
grouped_df.rename(columns={'LineTotal': 'TotalOrder'}, inplace=True)

# Mostrar las primeras filas
print(grouped_df.head(5).to_markdown(index=False, numalign="left", stralign="left"))

C:\Users\adria\AppData\Local\Temp\ipykernel_10176\1576760372.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


| SalesOrderID   | LineTotal   |
|:---------------|:------------|
| 43659          | 2024.99     |
| 43659          | 6074.98     |
| 43659          | 2024.99     |
| 43659          | 2039.99     |
| 43659          | 2039.99     |
| SalesOrderID   | TotalOrder   |
|:---------------|:-------------|
| 43659          | 20565.6      |
| 43660          | 1294.25      |
| 43661          | 32726.5      |
| 43662          | 28832.5      |
| 43663          | 419.459      |


In [30]:
#Filtra los productos que tienen un Color igual a 'Red' y guarda los resultados en un archivo CSV llamado "productos_rojos.csv".

# Consulta SQL para extraer los datos
query = "SELECT ProductID, color FROM Production.Product WHERE color LIKE 'Red%' "

# Leer datos en un DataFrame de Pandas
df = pd.read_sql(query, conn)

# Mostrar las primeras filas
print(df.head(5).to_markdown(index=False, numalign="left", stralign="left"))

# Guardar el DataFrame en un archivo CSV
df.to_csv('resultados_productos_Rojos.csv', index=False)  # index=False evita guardar el índice de las filas

| ProductID   | color   |
|:------------|:--------|
| 706         | Red     |
| 707         | Red     |
| 717         | Red     |
| 718         | Red     |
| 719         | Red     |


C:\Users\adria\AppData\Local\Temp\ipykernel_10176\1958768541.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [38]:
#Realiza un INNER JOIN entre las tablas Sales.SalesOrderHeader y Sales.Customer utilizando la columna CustomerID.
query = "SELECT CustomerID,PersonID FROM Sales.Customer "
query1= "SELECT CustomerID,SalesOrderID, OrderDate FROM Sales.Silver_SalesOrderHeader WHERE YEAR(OrderDate) = 2014"

# Leer los datos en un DataFrame de Pandas
df_sales_customer = pd.read_sql(query, conn) 
df_sales_order_header = pd.read_sql(query1,conn)
# Unión con FULL JOIN
df_merged_inner = pd.merge(df_sales_customer, df_sales_order_header, on='CustomerID', how='inner')

# Mostrar las primeras filas del DataFrame modificado (opcional)
print(df_merged_inner.head().to_markdown(index=False, numalign="left", stralign="left"))

| CustomerID   | PersonID   | SalesOrderID   | OrderDate   |
|:-------------|:-----------|:---------------|:------------|
| 11001        | 5454       | 72773          | 2014-05-12  |
| 11012        | 11663      | 68413          | 2014-03-17  |
| 11013        | 13873      | 74908          | 2014-06-23  |
| 11017        | 2521       | 68396          | 2014-03-16  |
| 11018        | 7394       | 69087          | 2014-03-26  |


C:\Users\adria\AppData\Local\Temp\ipykernel_10176\764379110.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sales_customer = pd.read_sql(query, conn)
C:\Users\adria\AppData\Local\Temp\ipykernel_10176\764379110.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sales_order_header = pd.read_sql(query1,conn)
